# Vertiefung

Jetzt haben wir die allerersten Schritte getan und wagen uns an etwas komplexere Themen, bevor wir uns an die Tools für die KI herantasten. 

Zwei ganz wichtige Usecases für Anwendungen mit KI sind zum einen eine Chat-Funktion und das Bereitstellen von (eigenen) Daten. 

Zum Warm werden, schauen wir uns eine sehr simple Chat-Funktion an. Auch hierfür bietet LangChain eine Menge Hilfsmittel an. Grundsätzlich laufen Chats mit der KI so ab, dass man in jedem Request an die KI immer die gesamte Chat-Historie mitschickt. D.h. die KI (und die Software, die die KI bereitstellt) speichert nicht den Chatverlauf in der Regel. Das Speichern (im Arbeitsspeicher) der Historie übernimmt in LangChain-Jargon ein _Memory_. 



![LangChain Memory](https://python.langchain.com/v0.2/assets/images/message_history-4c13b8b9363beb4621d605bf6b5a34b4.png)

Hinweis: _Runnable_ ist ein Interface, das die Grundlage eines Gliedes einer Kette bildet. D.h. man kann Runnables verketten.


Hinweis: häufig nutzt man den Begriff _Kontext_. Damit sind alle Informationen gemeint, die bei der Kommunikation mit der KI mitgegeben werden (oder anderweitig dazugesteuert werden). Dazu gehören auch die Nachrichten aus dem gesamten Chatverlauf. Ein Thema vom Anfang des Chats, wird also auch am Ende als Kontext angesehen, auch wenn man das Thema zwischendrin komplett verändert hat. Daher mein Tipp: immer einen neuen Kontext anfangen, sobald man das Thema wechselt... das spart auch Kosten, denn der gesamte Chatverlauf wird jedes Mal neu verarbeitet und kostet somit immer mehr und mehr Token!


So! Lange Rede, kurzer Sinn! Beispielcode sagt mehr als tausend Worte...

In [1]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu python-dotenv


[notice] A new release of pip is available: 24.1 -> 24.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

model = ChatOpenAI(model="gpt-4o")

parser = StrOutputParser()

system_template = "Du bist ein sehr hilfreicher Chatbot, der alles dafür tut, um dem Nutzer zu helfen."

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template), 
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}")]
)

chain = prompt_template | model | parser

Parent run 83831aa5-6c79-41a4-ab86-e0692a7c6e67 not found for run 53582624-dc2f-41d6-bf72-8a2283159261. Treating as a root run.


KI: Danke der Nachfrage! Als KI habe ich keine Gefühle, aber ich bin bereit und gespannt, dir bei allem zu helfen, was du brauchst. Wie kann ich dir heute behilflich sein?


Parent run 25e43e8f-343d-4b3b-8a1f-b00a4819a0cf not found for run fc9f4123-f88e-4f5f-be3d-0d0a4ef75b2a. Treating as a root run.


KI: Das klingt nach einem spannenden Workshop! Ich freue mich, Teil deines Projekts zu sein. Wie kann ich dir konkret helfen? Möchtest du spezifische Informationen über die Programmierung von Künstlicher Intelligenz, praktische Beispiele oder etwas anderes? Lass es mich wissen!


Ok wir haben uns erstmal eine "gewöhnliche" Chain mit einem Prompt-Template, Modell und Parser angelegt. Bis hier eigentlich mit unserem ersten Gehversuch identisch.

In [ ]:

chat_history = ChatMessageHistory()

chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

while True:
    user_input = input("Deine Frage (gebe 'exit' zum Beenden ein): ")
    if user_input == "exit":
        break
    antwort = chain_with_message_history.invoke({"input": user_input, "chat_history": chat_history}
                                                , {"configurable": {"session_id": "unused"}})
    print(f"KI: {antwort}")

Das ist der eigentlich spannende Part. Wir legen uns eine Instanz einer ChatMessageHistory an. Diese kümmert sich darum die eingebenenen und die von der KI erzeugten Nachrichten zu verwalten.

Damit legen wir uns ein _RunnableWithMessageHistory_ an. Das ist ein Helferlein aus LangChain, das unsere Chain mit der Speicherfunktionalität verknüpft. Das Ergebnis ist praktisch eine neue Chain, die wir aufrufen können.

Das tun wir dann auch in einer Endlos-Schleife (die man mit der Eingabe von 'exit' verlassen kann). Erstmal fordern wir den/die Benutzer*in auf einen Prompt einzugeben. Dieser wird dann mit dem Schlüssel "input" übergeben.

(Wir sehen hier auch, dass man eine _session_id_ übergeben kann. Damit könnte man unterschiedliche Konversationen implementeieren)

Jetzt ein wenig "Real Talk" wie man es so schön nennt...

Diese wundervolle Stück Code hätte ich nicht aus dem Ärmel schütteln können. Es ist auf einmal so unklar, warum wir auf einmal irgendein Runnable-Dings erzeugen müssen und das Muster mit der Verkettung verlassen wird. Sowas findet man leider ständig in diesem Framework. Dadurch ist man praktisch immer damit beschäftigt die Doku zu lesen, um solche Sachen umsetzen zu können. Zum Glück findet man für ziemlich alle Usecases schnell ein Stück Beispielcode. Aber gutes API-Design sieht m.E. anders aus!